# <center> OA3802 Comp Methods for Data Analytics: Lab 2 </center>
## <center> LT Jonathan Shepherd </center>
## <center> Email: jonathan.shepherd@nps.edu </center>

## Required Packages

In [1]:
# Package 
import os 
import pandas as pd
import xml.etree.ElementTree as ET
import re
import requests
from bs4 import BeautifulSoup

In [18]:
# Changing directory path
cor_path = "/Users/jonathanshepherd/OneDrive/NPS/Quarter7/OA3802_Comp_III/Labs/lab2_sakai/data"
os.chdir(cor_path)
# os.getcwd()

In [4]:
# Verifying that files are in the right path
path = os.getcwd()
# os.listdir(path) # Making sure I am in the correct directory with the right files

## Task 1: Inventory and Identify Files
<b>Answer:</b>

I used a series of Bash commands to identify the extensionless files.  

* File: source; Bash command:  xxd source | head -5; Hex signature: 25 50 44 46; file type: .pdf
* File: codel; Bash command: xxd codes | head -5; Hex signature: 3c 3f 78 6d; file type: .xml
* File: tracker; Bash command: xxd tracker | head -5; Hex signature: d0 cf 11 e0; file type: .xls 

The following website provides a more specific Hex signature match --> https://www.filesignatures.net/index.php?search=xml&mode=EXT

The tracker file had several different matches to include but not limited to doc, xls, ppt, and msg. The website mentioned above provides a more robust list. Even though the tracker file had multiple extension matches, a further examination of the file led me to believe that it's type was .xls.

After I figured out the correct file extensions, I changed them as as appropriate by conducting the next few steps:
<i>Note: These commands are specific to Mac OS.</i> 

1. Click on file
2. Press "Command" + "I" or Right-click and select "Get Info" on the keyboard.
3. Proceed to Name & Extension and add the appropriate extension
4. A prompt will appear asking me if I am sure that I want to add the extension. I will say yes. 

## Task 2: Extract and Enrich

<b>Process:</b> 
- After reading in the data. I used pandas to connect to grab the ISO3s from the xml file and attach it to the primary_data.csv. Unfortunately, a simple conditional matching was not enough to ensure that every country had their respective ISO3 attached. After my initial implementation, there were 25 country without a ISO3. Using regular expressions I was able to find the last 25 ISO3s. 
- Looking back on the assignment, I thought it would probably be much easier and cleaner to use <i>'is0_2'</i> as the key vice the <i>'country'</i> name. 

In [5]:
# Reading in codes.xml
tree = ET.parse('codes.xml')
root = tree.getroot()

# Reading in primary_data.csv
pri_data = pd.read_csv('primary_data.csv', keep_default_na=False)

In [6]:
iso3_dict = {}
iso3_dict_rev2 = {}
for country in root.findall('country'):
    name = country.get('name') # Grabbing the name of the country
    iso_3 = country.get('alpha-3') # Grabbing the ISO_3 information
    iso_2 = country.get('alpha-2') # Grabbing the ISO_3 information
    iso3_dict[name] = iso_3 
    iso3_dict_rev2[iso_2] = iso_3
#     print(name, iso_3)
# pri_data.columns

In [7]:
for keys, values in iso3_dict_rev2.items():
    #new_match = re.search(, keys)  
    pri_data.loc[pri_data.iso_2 == keys,'iso_3'] = values # Missing 25 values
#     print(keys,values)
#     print(pri_data.loc[pri_data.country != keys,'iso_3'])

In [8]:
no_iso3 = pri_data.loc[pri_data.iso_3.isnull() == True,'iso_3'].index # Finding the missing values; Missing 25 values 
unk_countries = pri_data.iloc[no_iso3,0] # States without iso_3s
# len(unk_countries)
# unk_countries # Checking to make sure I got all of the countries. 

In [9]:
# # Trying to grab the 25 remaining ISO3
# for i in unk_countries:
# #     print(i[0:5])
#     for keys, values in iso3_dict.items():
#         right_one = re.search(keys[0:4]+'.+', i)
# #         print(right_one)
#         if right_one:
# #             print(right_one.group())
#             # At this point is were I add the values 
# #             print(keys,right_one.group(),values)
#             pri_data.loc[pri_data.country == right_one.group(),'iso_3'] = values
#             break
#     for keys, values in iso3_dict.items():
#         right_one = re.search(keys[0:2]+'.+', i)
# #         print(right_one)
#         if right_one:
# #             print(right_one.group())
#             # At this point is were I add the values 
# #             print(right_one.group(),values)
#             pri_data.loc[pri_data.country == right_one.group(),'iso_3'] = values
#             break
# pri_data.loc[pri_data.country == "United States",'iso_3'] = "USA"

## Task 3: Scrape and Enrich: ITU Maritime Identification Digits     
<b>R Code: Code used to grab the data from the website. </b>.
- library(rvest)        
<br>
- #Specifying the url for desired website to be scraped    
- url <- "https://en.wikipedia.org/wiki/Maritime_identification_digits"       
<br>
- #Reading the HTML code from the website
- webpage <- read_html(url)    
<br>
- #Extract ALL tables from webpage    
- tables <- html_table(webpage, fill=TRUE)    
<br>
- #Extract and assign the sole table to the dataframe 'df'
- df <- tables[[1]]    
<br>
- str(df) # Checking the structure of the data. 
- head(df,10) # Seeing how the data looks.    
<br>
- #Extracting the MID file so I can read it into Python in order to present the transform the information into a useable product. 
- new.file <- write.csv(df, "~/OneDrive/NPS/Quarter7/OA3802_Comp_III/Labs/OA3802_Lab2/data//MID.csv",row.names = FALSE)


In [10]:
mid_df = pd.read_csv('MID.csv')

In [11]:
counter = 0
for i in mid_df.Country:
    for j in pri_data.country:
#         print(j,i)
        right_one = re.search(r""+j+"\s+\([A-z]+\s.*|"+j+'\s+\([A-z]+.*|'+j[0:3]+'[A-z]+\s.*|'+j[0:3]+'+.*|^C\S+\s[A-z].*|'+ j, i) 
        if right_one:
#                 print(mid_df.Country == i)
            mid_codes = ';'.join(mid_df.loc[mid_df.Country == i,"Codes"])#.astype(str)
#             print(type(mid_codes))
            if right_one.group()== i:
#                 print(right_one.group(), i)
#                 print(i,sum(pri_data.country ==right_one.group()),mid_codes, type(mid_codes))
                pri_data.loc[pri_data.country == j,"MID"] = mid_codes
                counter +=1 # Tracking to see how many matches I am getting 
#                 break
# counter
# pri_data.loc[pri_data.country == 'Cabo Verde',"MID"] = '306'
# pri_data.loc[pri_data.country == 'Sao Tome and Principe',"MID"] = '668'


In [12]:
pri_data.loc[pri_data.MID.isnull() == False,["country","iso_3","MID"]]
# pri_data.loc[pri_data.iso_3.isnull() == True,'iso_3'].index # Finding the missing values; No Missing values

,country,iso_3,MID
0,Indonesia,IDN,525
1,Panama,PAN,351; 352; 353; 354; 355; 356; 357; 370; 371; 3...
2,Japan,JPN,431; 432
3,China,CHN,270
4,United States,USA,338; 366; 367; 368; 369
...,...,...,...
172,Guinea,GIN,630
173,Zambia,ZMB,678
174,Macau,MAC,274
175,Uganda,UGA,675


## Task 4: Scrape and Enrich: Your Decision 
<b> R Code: Code used to pull suicide-rate data </b>    

- url <- "https://worldpopulationreview.com/country-rankings/suicide-rate-by-country
<br>
- #Reading the HTML code from the website
- webpage <- read_html(url)    
<br>
- #Extract ALL tables from webpage
- tables <- html_table(webpage, fill=F)    
<br>
- #Extract and assign the sole table to the dataframe 'df'
- df <- tables[[1]]    
<br>
- new.file <- write.csv(df, "~/OneDrive/NPS/Quarter7/OA3802_Comp_III/Labs/OA3802_Lab2/data//suicide_rate_by_country.csv",row.names = FALSE)    

<br>    
<b> Amplifying information: </b>       
- The suicide rate is measured at every 100,000. Ex: Belgium, which ranks at number eleven with 20.7 suicides per 100, 100


In [13]:
suicide_df = pd.read_csv('suicide_rate_by_country.csv')

In [14]:
# Changing column names
pri_data.rename(columns={"country":"Country"}, inplace=True)# I needed to rename one of the columns, to match the other, so the merge can take place
suicide_df = suicide_df.rename(columns={"Suicide Rank":"Suicide_Rank","Total Suicide Rate":"Total_Suicide_Rate", "Male Suicide Rate":"Male_Suicide_Rate","Female Suicide Rate":"Female_Suicide_Rate","Total Per Year":"Total_Per_Year" })

# Changin the type of the data to do math 
suicide_df["Total_Per_Year"] = suicide_df["Total_Per_Year"].replace(r',','', regex=True).astype(float)
suicide_df.sort_values(by=['Suicide_Rank']). head(11)

,Country,Suicide_Rank,Total_Suicide_Rate,Male_Suicide_Rate,Female_Suicide_Rate,Total_Per_Year
0,Lithuania,1,31.9,58.1,9.5,894.0
1,Russia,2,31.0,55.9,9.4,45178.0
2,Guyana,3,29.2,43.7,14.4,227.0
3,South Korea,4,26.9,38.4,15.4,13765.0
4,Belarus,5,26.2,46.9,8.2,2477.0
5,Suriname,6,22.8,34.7,10.9,131.0
6,Kazakhstan,7,22.5,38.3,7.6,4122.0
7,Ukraine,8,22.4,41.1,6.3,9911.0
8,Latvia,9,21.2,37.6,7.3,409.0
9,Lesotho,10,21.2,17.8,24.4,447.0


In [15]:
pri_data = pd.merge(pri_data, suicide_df, how='left',on="Country")
# pri_data
# suicide_df.loc[suicide_df.Country == "Laos",]
# pri_data.tail(10)
# pri_data.head()

In [16]:
# Grabing the rest of the data
counter = 0
for i in suicide_df.Country:
    for j in pri_data.Country:
#         print(j,i) Kor\S+\s[A-z].*
        right_one = re.search(r""+j[0:3]+'\S+\s[A-z].*|'+j[0:3]+'\S+\s|'+j[0:3]+'\S+\,\s[A-z].*', i) 
#         print(j[0:3])
        if right_one:
            Suicide_Rank = ';'.join(suicide_df.loc[suicide_df.Country == i, "Suicide_Rank"].astype(str))
            Total_Suicide_Rate = ';'.join(suicide_df.loc[suicide_df.Country == i, "Total_Suicide_Rate"].astype(str))
            Male_Suicide_Rate = ';'.join(suicide_df.loc[suicide_df.Country == i, "Male_Suicide_Rate"].astype(str))
            Female_Suicide_Rate = ';'.join(suicide_df.loc[suicide_df.Country == i, "Female_Suicide_Rate"].astype(str))
            Total_Per_Year = ';'.join(suicide_df.loc[suicide_df.Country == i, "Total_Per_Year"].astype(str))
            
#             print(Suicide_Rank,Total_Suicide_Rate,Male_Suicide_Rate,Female_Suicide_Rate,Total_Per_Year)
            if right_one.group()== i:
#                 print(right_one.group())
                pri_data.loc[pri_data.Country == j,"Suicide_Rank"] = Suicide_Rank
                pri_data.loc[pri_data.Country == j,"Total_Suicide_Rate"] = Total_Suicide_Rate
                pri_data.loc[pri_data.Country == j,"Male_Suicide_Rate"] = Male_Suicide_Rate
                pri_data.loc[pri_data.Country == j,"Female_Suicide_Rate"] = Female_Suicide_Rate
                pri_data.loc[pri_data.Country == j,"Total_Per_Year"] = Total_Per_Year
                counter +=1 # Tracking to see how many matches I am getting 
#                break

### Merged Suicide data with primary data frame. 

In [17]:
# pri_data.loc[pri_data.Suicide_Rank.isnull() == False,["Country","iso_3","Suicide_Rank"]]#.shape[0] # I Properly matched 141
# pri_data.loc[pri_data.Suicide_Rank.isnull() == True,["Country","iso_3","Suicide_Rank"]].shape[0]  # Missing 29
pri_data

,Country,merchant_vessel_count,iso_2,iso_3,MID,Suicide_Rank,Total_Suicide_Rate,Male_Suicide_Rate,Female_Suicide_Rate,Total_Per_Year
0,Indonesia,8782,ID,IDN,525,158,3.4,4.8,2,9101
1,Panama,8052,PA,PAN,351; 352; 353; 354; 355; 356; 357; 370; 371; 3...,148,4.3,7.4,1.2,180
2,Japan,5289,JP,JPN,431; 432,14,18.5,26,11.4,23532
3,China,4287,CN,CHN,270,69,9.7,9.1,10.3,138482
4,United States,3611,US,USA,338; 366; 367; 368; 369,170,2.8,3.6,0.7,270.0
...,...,...,...,...,...,...,...,...,...,...
172,Guinea,1,GN,GIN,630,151,4.0,4.7,3.3,75.0
173,Zambia,1,ZM,ZMB,678,116,6.1,8.8,3.5,1058
174,Macau,1,MO,MAC,274,NaN,NaN,NaN,NaN,NaN
175,Uganda,1,UG,UGA,675,68,9.9,10.7,9.1,4230


## Task 4: Scrape and Enrich: Your Decision
<b> Process: </b>    
- Using a function I created in R, I was able to extract the appropriate AIS files (01-03Nov2018) called <b> AIS_file_producer</b> (Ref: AIS_file_producter.R). This function has the ability to create a text file with the requested dates and put them in the appropriate directory. 
- I then used the Bash command <b><i> wget -i AIS2018-11-01.txt </i></b> to grab the three zip files. 
- I opened the zip files using the command <b><i> wget unzip '*.zip'</i></b> 

<center>Country Name</center>    |<center> MMSI Codes</center>         
:---------------|:----------------------------------------------------------: 
 Panama         | 351; 352; 353; 354; 355; 356; 357; 370; 371; 372; 373; 374 
Marshall Islands| 538 
Liberia         | 636; 637
Hong Kong       | 477
Singapore       | 563; 564; 565; 566

In [21]:
# Changing file directory 
os.getcwd()

os.chdir(cor_path+"/AIS_Data")
# os.listdir(os.getcwd())
# os.getcwd()

In [22]:
AIS_file = pd.read_csv('AIS_2018_11_01.csv', keep_default_na=False)
mmsi_pings_dict = {}
panama_mmsi_pings_list = []
mi_mmsi_pings_list = []
liberia_mmsi_pings_list = []
hk_mmsi_pings_list = []
singapore_mmsi_pings_list = []

In [23]:
i = '35145343234'
panama_mmsi = re.search(r'^351.+|^352.+|^353.+|^354.+|^355.+|^356.+|^357.+|^370.+|^371.+|^372.+|^373.+|^374.+', i)
panama_mmsi

<re.Match object; span=(0, 11), match='35145343234'>

In [24]:
for i in AIS_file['MMSI'].astype(str):
#     print(type(i))
    # Creating Regex to find countries
    panama_mmsi = re.search(r'^351.+|^352.+|^353.+|^354.+|^355.+|^356.+|^357.+|^370.+|^371.+|^372.+|^373.+|^374.+', i) 
    mi_mmsi = re.search(r'^538.+', i) 
    liberia_mmsi = re.search(r'^636.+|^637.+', i) 
    hk_mmsi = re.search(r'^477.+', i) 
    singapore_mmsi = re.search(r'^563.+|^564.+|^565.+|^566.+', i) 

    if panama_mmsi:
        panama_mmsi_pings_list.append(int(panama_mmsi.group()))
#             print(panama_mmsi.group())
        mmsi_pings_dict['Panama'] = panama_mmsi_pings_list

    elif mi_mmsi:
        mi_mmsi_pings_list.append(int(mi_mmsi.group()))
        mmsi_pings_dict['Marshall_Islands'] = mi_mmsi_pings_list

    elif liberia_mmsi:
        liberia_mmsi_pings_list.append(int(liberia_mmsi.group()))
        mmsi_pings_dict['Liberia'] = liberia_mmsi_pings_list

    elif hk_mmsi:
        hk_mmsi_pings_list.append(int(hk_mmsi.group()))
        mmsi_pings_dict['Hong_Kong'] = hk_mmsi_pings_list    


    elif singapore_mmsi:
        singapore_mmsi_pings_list.append(int(singapore_mmsi.group()))
        mmsi_pings_dict['Singapore'] = singapore_mmsi_pings_list


In [25]:
# This functions grabs the data for the countries listed above. 
def countries_count(AIS_file):    
    mmsi_pings_dict = {}
    panama_mmsi_pings_list = []
    mi_mmsi_pings_list = []
    liberia_mmsi_pings_list = []
    hk_mmsi_pings_list = []
    singapore_mmsi_pings_list = []


    for i in AIS_file['MMSI'].astype(str):
    #     print(type(i))
        # Creating Regex to find countries
        panama_mmsi = re.search(r'^351.+|^352.+|^353.+|^354.+|^355.+|^356.+|^357.+|^370.+|^371.+|^372.+|^373.+|^374.+', i) 
        mi_mmsi = re.search(r'^538.+', i) 
        liberia_mmsi = re.search(r'^636.+|^637.+', i) 
        hk_mmsi = re.search(r'^477.+', i) 
        singapore_mmsi = re.search(r'^563.+|^564.+|^565.+|^566.+', i) 

        if panama_mmsi:
            panama_mmsi_pings_list.append(int(panama_mmsi.group()))
#             print(panama_mmsi.group())
            mmsi_pings_dict['Panama'] = panama_mmsi_pings_list

        if mi_mmsi:
            mi_mmsi_pings_list.append(int(mi_mmsi.group()))
            mmsi_pings_dict['Marshall_Islands'] = mi_mmsi_pings_list

        if liberia_mmsi:
            liberia_mmsi_pings_list.append(int(liberia_mmsi.group()))
            mmsi_pings_dict['Liberia'] = liberia_mmsi_pings_list

        if hk_mmsi:
            hk_mmsi_pings_list.append(int(hk_mmsi.group()))
            mmsi_pings_dict['Hong_Kong'] = hk_mmsi_pings_list    


        if singapore_mmsi:
            singapore_mmsi_pings_list.append(int(singapore_mmsi.group()))
            mmsi_pings_dict['Singapore'] = singapore_mmsi_pings_list
            
    return(mmsi_pings_dict)

# ais_pd = pd.DataFrame(mmsi_pings_dict)

In [27]:
required_data_list = [] 
# AIS = 'f'
required_files = ['AIS_2018_11_01.csv','AIS_2018_11_02.csv','AIS_2018_11_03.csv']
for files in required_files:
    AIS = pd.read_csv(files, keep_default_na=False)
    AIS_data = countries_count(AIS)
    required_data_list.append(AIS_data)

In [28]:
rd_list = ["Panama", "Marshall_Islands", "Liberia", "Hong_Kong", "Singapore"]
prid_list = ["Panama", "Marshall Islands", "Liberia", "Hong Kong", "Singapore"]
activity_ratio_dict = {}
counter = 0

for i in rd_list:
#     print(i)
    if i not in activity_ratio_dict:
#         print(i)
#         [0] == 2018_11_01; [1] == 2018_11_02; [2] == 2018_11_03
#         print(len(required_data_list[0][i])+ len(required_data_list[1][i])+len(required_data_list[2][i]))
        
        ac_ration = (len(required_data_list[0][i])+ \
                     len(required_data_list[1][i])+ \
                     len(required_data_list[2][i])) 
        ac_ration = ac_ration / pri_data.loc[pri_data.Country == prid_list[counter],"merchant_vessel_count"]
#         print(type(prid_list[counter]))
        activity_ratio_dict[i] = round(ac_ration.values[0],3) # .values[0] gets ride of all of the extra panda wording.
        counter +=1
activity_ratio_dict    

{'Panama': 43.609,
 'Marshall_Islands': 116.104,
 'Liberia': 76.21,
 'Hong_Kong': 50.991,
 'Singapore': 40.438}

In [29]:
rd_list = ["Panama", "Marshall_Islands", "Liberia", "Hong_Kong", "Singapore"]
prid_list = ["Panama", "Marshall Islands", "Liberia", "Hong Kong", "Singapore"]
vessel_ratio_dict = {}
counter = 0

for i in rd_list:
#     print(i)
    if i not in vessel_ratio_dict:
#         print(i)
        # unique ratio 
#         print(len(required_data_list[0][i])+ len(required_data_list[1][i])+len(required_data_list[2][i]))
        vess_ration = (len(pd.DataFrame(required_data_list[0][i])[0].unique())+ \
                     len(pd.DataFrame(required_data_list[1][i])[0].unique()) + \
                     len(pd.DataFrame(required_data_list[2][i])[0].unique()))
#         print(vess_ration)
        vess_ration = vess_ration / pri_data.loc[pri_data.Country == prid_list[counter],"merchant_vessel_count"]
#         print(type(prid_list[counter]))
#         print(type(uniq_ration))
        vessel_ratio_dict[i] = round(vess_ration.values[0],3)
        counter +=1

# vessel_ratio_dict

In [30]:
for keys, values in activity_ratio_dict.items():
    print('Country:', keys)
    print('\t\t Activity Ratio: ', values)
    print('\t\t Unique Ratio: ', vessel_ratio_dict[keys])

Country: Panama
		 Activity Ratio:  43.609
		 Unique Ratio:  0.104
Country: Marshall_Islands
		 Activity Ratio:  116.104
		 Unique Ratio:  0.287
Country: Liberia
		 Activity Ratio:  76.21
		 Unique Ratio:  0.177
Country: Hong_Kong
		 Activity Ratio:  50.991
		 Unique Ratio:  0.12
Country: Singapore
		 Activity Ratio:  40.438
		 Unique Ratio:  0.104
